In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# import sys
# sys.path.insert(0, '../') # you should not need to uncomment this!
# import warnings
# warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [ ]:
import torch
from pathlib import Path
import matplotlib.pyplot as plt

from poutyne import Model, SKLearnMetrics
from sklearn.metrics import r2_score, explained_variance_score, mean_squared_error, median_absolute_error
from mlflood.dataset import load_dataset
from mlflood.models.CNNrolling import CNNrolling
from mlflood.models.unet import UNet
from mlflood.utils import metric_flatten, get_poutyne_callbacks, saferm
from torchsummary import summary
import numpy as np

In [ ]:
cuda_device = 0
device = torch.device("cuda:%d" % cuda_device if torch.cuda.is_available() else "cpu")

In [ ]:
catchment_num = "toy"
experiment_name = "test"
border_size = 0

# parameter for the catchment
catchment_kwargs = {}
catchment_kwargs["tau"]=0.5
catchment_kwargs["timestep"]=3                # for timestep >1 use CNN rolling or Unet
catchment_kwargs["sample_type"]="full"
catchment_kwargs["dim_patch"]=60
catchment_kwargs["fix_indexes"]=False
catchment_kwargs["border_size"] = border_size

# optimization paramters
optimization_kwargs = {}
optimization_kwargs["batch_size"] = 8
optimization_kwargs["epochs"] = 2             



In [ ]:
inp_dim = 1 + catchment_kwargs["timestep"]*2
print('Input dim: ', inp_dim)

In [ ]:
network = CNNrolling(border_size=border_size, timestep = catchment_kwargs["timestep"])
# summary(network, [(inp_dim, 120,120), (1,120,120)])

In [ ]:
def get_l1_loss_weight(output, target):
    mask = output[:,1]
    predictions =  output[:,0]
    target = target[:,0]
    predictions = torch.mul(predictions, mask)
    loss = torch.abs(predictions  - target)
#     a = target > 0.2
#     loss[a] = loss[a] *4
    loss = loss.sum()/(torch.sum(mask.float()))
    return loss   # comput the mean only considering elemnts in the mask


In [ ]:
model = Model(network, 'adam', get_l1_loss_weight,
              batch_metrics=["l1"],
              epoch_metrics=[ SKLearnMetrics(metric_flatten(r2_score)), 
                              SKLearnMetrics(metric_flatten(explained_variance_score)), 
                              SKLearnMetrics(metric_flatten(mean_squared_error)), 
                              SKLearnMetrics(metric_flatten(median_absolute_error))
                            ],
              device=device)

In [ ]:
# Get callbacks for tensorboard and saving the model
callbacks, summary_dir, checkpoint_dir = get_poutyne_callbacks(experiment_name)

In [ ]:
train_dataset, valid_dataset = load_dataset(catchment_num=catchment_num, **catchment_kwargs)

In [ ]:
plt.imshow(train_dataset.dem)
plt.colorbar()

In [ ]:
saferm(summary_dir)
saferm(checkpoint_dir)
summary_dir.mkdir(parents=True, exist_ok=True)
checkpoint_dir.mkdir(parents=True, exist_ok=True)

history = model.fit_dataset(train_dataset, valid_dataset=valid_dataset, **optimization_kwargs, callbacks=callbacks) 

In [ ]:
e = [v['epoch'] for v in history]
val_loss = [v['val_loss'] for v in history]
train_loss = [v['loss'] for v in history]
plt.plot(e, train_loss, label="Training MSE")
plt.plot(e, val_loss, label="Validation MSE")
plt.xlabel("Epochs")
plt.legend()

### Mse for CNN

In [ ]:
dataset = valid_dataset

In [ ]:
from mlflood.evaluation import pred_1step, comp_mse_ag

predictions_ag = pred_1step(model, dataset)

timestep = catchment_kwargs["timestep"]

mse_roll = comp_mse_ag(predictions_ag)
for event_n in range(len(predictions_ag)):
    mse_roll[event_n] = np.insert(mse_roll[event_n], 0, [0]*(timestep-1))

t = np.arange(1,len(dataset.rainfall_events[0]))

plt.gca().set_prop_cycle(plt.cycler('color', plt.cm.jet(np.linspace(0, 2,2))))
plt.plot(t, mse_roll[0].T, "-", label="Rolling CNN event 1")
plt.plot(t, mse_roll[1].T, "-", label="Rolling CNN event 2")

plt.xlabel("Time [step]")
plt.ylabel("MSE")
plt.legend()

### MSE for comparing 1 step prediction of current model with baseline 

In [ ]:
from mlflood.evaluation import pred_1step, comp_mse_ag, base_model

dataset = valid_dataset

predictions_ag = pred_1step(base_model(), dataset)
base_mse = comp_mse_ag(predictions_ag)
for event_n in range(len(predictions_ag)):
    base_mse[event_n] = np.insert(base_mse[event_n], 0, [0]*(timestep-1))
base_mse = np.array(base_mse)


predictions_ag = pred_1step(model, dataset)
mse = comp_mse_ag(predictions_ag)
for event_n in range(len(predictions_ag)):
    mse[event_n] = np.insert(mse[event_n], 0, [0]*(timestep-1))
mse = np.array(mse)
    

t = np.arange(1,len(dataset.rainfall_events[0]))

plt.gca().set_prop_cycle(plt.cycler('color', plt.cm.jet(np.linspace(0, 2,2))))
plt.plot(t, base_mse.T, "--", label="Baseline")
plt.plot(t, mse.T, "-", label="CNN model")

plt.xlabel("Time [step]")
plt.ylabel("MSE")
plt.legend()

### Mse only in Autoregressive mode (timestep=1)

In [ ]:
from mlflood.evaluation import pred_ag, comp_mse_ag, base_model

predictions_ag = pred_ag(base_model(), dataset)
base_mse = np.array(comp_mse_ag(predictions_ag))

predictions_ag = pred_ag(model, dataset)
mse = np.array(comp_mse_ag(predictions_ag))

t = np.arange(dataset.timestep,len(dataset.rainfall_events[0]))

plt.gca().set_prop_cycle(plt.cycler('color', plt.cm.jet(np.linspace(0, 2,2))))
plt.plot(t, base_mse.T, "--", label="Baseline")
plt.plot(t, mse.T, "-", label="Base CNN")

plt.xlabel("Time [step]")
plt.ylabel("MSE")
plt.legend()



### Display current model result

In [ ]:
from mlflood.evaluation import numpy2movie
predictions_ag = pred_1step(model, dataset)               # use pred_ag for prediction in autoregressive model

animation = numpy2movie(*predictions_ag[0])
animation.ipython_display(fps=10, loop=True, autoplay=True)